# "Naiver" Client

Wir kontaktieren unseren eigenen lokalen Chatserver. Seine IP-Adresse ist eine Adresse innerhalb des Docker-Netzwerks. Vorerst verwenden wir 127.0.0.1. Dafür können wir auch `localhost` schreiben. (Wie Sie miteinander chatten können, verrate ich Ihnen später) Als Port für Anfragen verwenden wir den (freien) Port 5000.
Außerdem legen wir hier schon fest, wie der Client heißt (später werden wir weitere Clients erzeugen, die wir am Benutzername unterscheiden)

Es kann passieren, dass der primitive Server auf falsche Reihenfolgen mit Blockaden reagiert. Sollte dies der Fall sein, stoppen Sie ihn über die `Shut Down all Kernel Sessions` (zweiter Eintrag in der linken Symbolleiste) und starten Sie ihn wieder neu. Restarten Sie dann den Kernel für dieses Notebook und wiederholen Sie die Schritte zur Anmeldung.

Ggf. müssen Sie etwas abwarten (Adress already in use) oder Ihren Jupyterserver neu starten. Wenn oben rechts "no kernel" steht, müssen Sie `Restart the Kernel` ausführen.

### Wir fangen ganz einfach an und melden und zunächst mal beim Server an.

In [1]:
import socket

# Wir erzeugen ein Verbindungselement für IPv4-Kommunikation (AF_INET) und TCP (SOCK_STREAM), also eine "klassische" TCP/IP Verbindung. Damit man die Zahlen nicht auswendig lernen muss 
# definiert das Modul Socket die Konstanten
print(f'Beispiel: Der Zahlenwert für {str(socket.AF_INET)} ist {int(socket.AF_INET)}')

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

Beispiel: Der Zahlenwert für AddressFamily.AF_INET ist 2


In [2]:
host = ('localhost', 5001)  # Bitte Ihre Ilias-Kennung eintragen
BUFSIZ = 1024 # Größe des Puffers. Soll größer sein als die erwartete maximale Länge eines Posts

In [3]:
# Verbindungsanfrage an den Server
client_socket.connect(host)
answer = client_socket.recv(BUFSIZ).decode()
# Der Server antwortet sofort mit der Aufforderung, den Benutzernamen einzugeben
print(answer)

Bitte Namen eingeben


In der Datenübertragung kennt man keine Strings sondern nur Folgen von Bytes. Vor dem Senden mit `socket.send()` muss ein String deshalb nach festgelegter Umwandlungstabellen in eine Zahlenfolge gewandelt werden. Diesen Prozess nennt man "Kodierung", englisch "encoding". Wir verwenden künftig konsequent die Methode `.encode('utf-8')` eines String-Objekts zur Kodierung. UTF-8 ist die heute am meisten verbreitete Kodiertabelle und Standard für Python-Strings. Deshalb darf man den Parameter auch weglassen.
 

Das Lesen einer Server-Antwort geschieht mit `socket.recv(BUFSIZE)` (Bufsize ist die Größe eines Puffers für die empfangenen Bytes). Das Ergebnis von `recv` ist ein Objekt des Typs `bytes`, dessen Methode `.decode('utf-8')` zur Dekodierung also zur Umwandlung in `string` benutzt werden kann. Dadurch können wir im Python-Programmen ganz normal mit Strings arbeiten. 
Beachten Sie das 'b' vor dem Anführungsstrich in der Serverantwort. Wir erkennen daran, dass der Server uns eine bytes-Folge geliefert hat

In [4]:
# Dies ist ein String
answer.decode('utf-8')
# Auf die Angabe der Kodiertabelle kann man in einfachen Fällen (keine Umlaute etc.) verzichten

AttributeError: 'str' object has no attribute 'decode'

In [5]:
# Nun sollten wir den Server mit dem Benutzernamen versorgen und damit die Anmeldung abschließen
name = input('Bitte Benutzername eingeben: ')
client_socket.send(name.encode())
# Der Server antwortet auf jeden send mit einer Antwort. Diese müssen wir abholen und lesen
answer = client_socket.recv(BUFSIZ).decode()
print(answer)

Bitte Benutzername eingeben:  Mambo


Willkommen Mambo! Zum Verlassen "<ende>" eingeben.


### Anmeldung abgeschlossen. Ab jetzt erhalten wir für jede Botschaft eine Antwort

In [6]:
client_socket.send('mein erste Posting'.encode())

18

In [7]:
# Anwort lesen, dekodieren und ausgeben. 
print(client_socket.recv(BUFSIZ).decode())

Mambo: mein erste Posting


In [ ]:
# Für die richtige Reihenfolge sorgt diese Schleife
msg = ''
while msg != '<ende>':
    msg = input('Botschaft:')
    client_socket.send(msg.encode())
    print(client_socket.recv(BUFSIZ).decode())

Botschaft: Hallo, das ist meine zweite Botschaft


Mambo: Hallo, das ist meine zweite Botschaft


Botschaft: hallo


Mambo: hallo


Botschaft: März


Mambo: März


Botschaft: Hallo


Mambo: Hallo


### Und wo ist jetzt das Problem?

So lange Sie mit sich selbst chatten, gibt es kein Problem. Aber der Spass ist nur von begrenzter Dauer. Sobald die Symmetrie Frage-Antwort verletzt wird, funktioniert das Programm nicht mehr so, wie es soll. Machen Sie einen einfachen Test: Kommentieren Sie einfach die Anweisung in Zelle 7 `print(client_socket.recv(BUFSIZ).decode())` aus. Sie erhalten nicht mehr das Echo auf Ihre Eingabe, sonderns das des zurückliegenden `send`. Wenn Sie nun auch noch die `client_socket.send()`-Anweisung aktivieren sehen Sie sehr schön die Funktion einer `Pipeline`. Alle `send`-Anweisungen schicken Bytes auf die Reise. Mit `recv` holt man (bis zur Größe BUFSIZ) alles ab, was aufgelaufen ist.

Wenn nun ein weiterer Client dazukommt, ist die Symmetrie endgültig gestört. Probieren Sie es auch. Duplizieren Sie dieses Notebook, ändern Sie den Usernamen und spielen Sie mal.